In [1]:
from environments.gridworld import GridWorld
from lonr.local_no_regret import LocalNoRegret

In [4]:
grid_world = GridWorld()

TypeError: 'State' object is not iterable

In [ ]:
a = [(1,2), (3,4)]
for x in a:
